<a href="https://colab.research.google.com/github/deayalar/deeplearning_unitn/blob/develop/Copy_of_DL_project_new_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !wget https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
# !unzip -q dataset.zip -d dataset

# !rm -rf /content/deeplearning_unitn
# !git clone https://github.com/deayalar/deeplearning_unitn.git

# !nvidia-smi

# %cd /content/deeplearning_unitn

In [2]:
%cd /content/deeplearning_unitn
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from cost_functions import cross_entropy
import cost_functions
from evaluation import Evaluator
from my_datasets.reid_dataset import Market1501
from utils.split_data import ValidationSplitter, TrainingSplitter
from models.reid_model import ReIdModel, FinetunedModel
from sklearn.model_selection import train_test_split

/content/deeplearning_unitn


In [3]:
config = dict(
    wandb = False,
    device = "auto", # Select an specific device None to select automatically
    train_root = "/content/dataset/train",
    test_root = "/content/dataset/test",
    queries_root = "/content/dataset/queries",
    attributes_file = "/content/dataset/annotations_train.csv",
    # train_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/train",
    # test_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/test",
    # queries_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/queries",
    # attributes_file = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/annotations_train.csv",
    dataset="Market1501",
    backbone = "resnet18",
    loss = cross_entropy(),
    split = dict(
        full_training_size = 0.75,
        train_size = 0.8
    ),
    compose = dict(
        resize_h = 224,
        resize_w = 224
    ),
    epochs=1,
    training_batch_size=8,
    validation_batch_size=32,
    learning_rate=0.01,
    weight_decay=0.000001, 
    momentum=0.9, 
    mAP_rank=10)

In [4]:
def split_data(config):
    print('split_data OK!')
    splitter = ValidationSplitter(train_root=config["train_root"], 
                                  test_root=config["test_root"], 
                                  queries_root=config["queries_root"])
    train_set, val_set, val_queries = splitter.split(train_size=config["split"]["full_training_size"],
                                                     random_seed=42)
    return train_set, val_set, val_queries
train_set, val_set, val_queries = split_data(config)

split_data OK!
Extract queries proportion: 0.11
Identities in train set: 563
Identities in validation set: 188
Train set size: 9635
Validation set size: 2985
Number of validation queries: 369


In [5]:
from my_datasets.reid_dataset import Market1501

#Create pytorch Datasets 
composed = transforms.Compose([transforms.Resize((config["compose"]["resize_h"], 
                                                  config["compose"]["resize_w"])),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.224, 0.225])])

train_dataset = Market1501(root_dir=config["train_root"],
                        attributes_file=config["attributes_file"],
                        #full_train_set=full_train_set,
                        images_list=train_set,
                        transform=composed)

In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                            batch_size=config["training_batch_size"], 
                                            shuffle=True, 
                                            # num_workers=8
                                            num_workers=4)
                                            
# val_loader = torch.utils.data.DataLoader(val_dataset, 
#                                             batch_size=config["validation_batch_size"], 
#                                             shuffle=False, 
#                                             num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
# # import pandas as pd
# # import numpy as np
# # attr = pd.read_csv('/content/dataset/annotations_train.csv')
# # print(len(attr))
# # #for column in attr.columns:
# # #  print('column: ', max(attr[column]))

# for column in self.attr_df.columns:
#   if(column!='age'):
#     self.attr_df[column] =np.array(self.attr_df[column].astype('str').replace({'1': '0', '2': '1'}))
# # attr.head(20)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models

PRETRAINED_MODELS = {
    "resnet18": { "load": lambda : models.resnet18(pretrained=True), "feature_size": 512},
    "resnet50": { "load": lambda : models.resnet50(pretrained=True), "feature_size": 2048}
    # More pretrained models here e.g. alexnet, vgg16, etc
}

class FinetunedModel(nn.Module):
    def __init__(self, architecture, n_classes):
        super(FinetunedModel, self).__init__()
        self.architecture = architecture

        self.backbone = PRETRAINED_MODELS[architecture]["load"]()
        self.feature_size = PRETRAINED_MODELS[architecture]["feature_size"]
        print(f"Backbone feature size: {self.feature_size}")
        self.finetune(self.backbone, n_classes)

    def finetune(self, model, n_classes):
        model_name = model.__class__.__name__
        if model_name.lower().startswith("resnet"):
            self.features =   nn.Sequential(*list(model.children())[:-1])
            #we are using output as 1 because we are going to use binary classifcation
            self.age_classifier        =   nn.Sequential(nn.Linear(self.feature_size,1),
                                                         nn.Sigmoid()) #4
            self.backpack_classifier   =   nn.Sequential(nn.Linear(self.feature_size,1),
                                                         nn.Sigmoid())
            self.bag_classifier        =   nn.Sequential(nn.Linear(self.feature_size,1),
                                                         nn.Sigmoid())
            # self.handbag_classifier    =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.clothes_classifier    =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.down_classifier       =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.up_classifier         =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.hair_classifier       =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.hat_classifier        =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.gender_classifier     =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.upblack_classifier    =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.upwhite_classifier    =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.upred_classifier      =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.uppurple_classifier   =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.upyellow_classifier   =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.upgray_classifier     =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.upblue_classifier     =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.upgreen_classifier    =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.downblack_classifier  =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.downwhite_classifier  =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.downpink_classifier   =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.downpurple_classifier =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.downyellow_classifier =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.downgray_classifier   =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.downblue_classifier   =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.downgreen_classifier  =   nn.Sequential(nn.Linear(self.feature_size,1))
            # self.downbrown_classifier  =   nn.Sequential(nn.Linear(self.feature_size,1))

    def forward(self, x, get_features=False):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        if get_features:
            return [x]
        
        # age                   = self.age_classifier(x)
        backpack_classifier   = self.backpack_classifier(x)
        bag_classifier        = self.bag_classifier(x)  
        # handbag_classifier    = self.handbag_classifier(x)  
        # clothes_classifier    = self.clothes_classifier(x)  
        # down_classifier       = self.down_classifier(x)     
        # up_classifier         = self.up_classifier(x)      
        # hair_classifier       = self.hair_classifier(x)     
        # hat_classifier        = self.hat_classifier(x)      
        # gender_classifier     = self.gender_classifier   
        # upblack_classifier    = self.upblack_classifier(x) 
        # upwhite_classifier    = self.upwhite_classifier(x) 
        # upred_classifier      = self.upred_classifier(x)    
        # uppurple_classifier   = self.uppurple_classifier(x) 
        # upyellow_classifier   = self.upyellow_classifier(x) 
        # upgray_classifier     = self.upgray_classifier(x)   
        # upblue_classifier     = self.upblue_classifier(x)   
        # upgreen_classifier    = self.upgreen_classifier(x)  
        # downblack_classifier  = self.downblack_classifier(x)  
        # downwhite_classifier  = self.downwhite_classifier(x)  
        # downpink_classifier   = self.downpink_classifier(x)  
        # downpurple_classifier = self.downpurple_classifier(x) 
        # downyellow_classifier = self.downyellow_classifier(x) 
        # downgray_classifier   = self.downgray_classifier(x)   
        # downblue_classifier   = self.downblue_classifier(x)   
        # downgreen_classifier  = self.downgreen_classifier(x)  
        # downbrown_classifier  = self.downbrown_classifier(x) 
        return backpack_classifier, bag_classifier 
                #backpack_classifier,
                #bag_classifier
                # handbag_classifier,
                # clothes_classifier,
                # down_classifier,
                # up_classifier,
                # hair_classifier,
                # hat_classifier,
                # gender_classifier,
                # upblack_classifier,
                # upwhite_classifier,
                # upred_classifier,
                # uppurple_classifier,
                # upyellow_classifier,
                # upgray_classifier,
                # upblue_classifier,
                # upgreen_classifier,
                # downblack_classifier,
                # downwhite_classifier,
                # downpink_classifier,
                # downpurple_classifier,
                # downyellow_classifier,
                # downgray_classifier,
                # downblue_classifier,
                # downgreen_classifier,
                # downbrown_classifier
                #]



In [9]:
# net = FinetunedModel("resnet18", 27)
# iterator = iter(train_loader)
# x, ids, attrs = next(iterator)
# print(x.size())

# output = net(x,get_features=False)
# print(output[0].size())

# print(output)


In [25]:
'''
Let's take the backbone. weights trained. 

'''


def initialize_resnet18(num_classes):
    model_conv = torchvision.models.resnet18(pretrained=True)

    num_ftrs = model_conv.fc.in_features
    model_conv.fc = nn.Linear(num_ftrs, num_classes)
    return model_conv


class MultiTaskLossWrapper(nn.Module):
    def __init__(self, task_num):
        super(MultiTaskLossWrapper, self).__init__()
        self.task_num = task_num
        #self.log_vars = nn.Parameter(torch.zeros((task_num)))

    def forward(self, preds, attrs):

        #bce, crossEntropy = nn.BCELoss(), CrossEntropyFlat()
        bce = nn.BCELoss()
        # mock = torch.empty(8, 1).random_(2)

        #print(attrs[:,1].unsqueeze(1).size())
        #print(attrs[:,1].unsqueeze(1))
        # loss_backpack = bce(preds[1], mock.to("cuda:0"))
        loss_backpack = bce(preds[1], attrs[:,1].unsqueeze(1).to(torch.float32))
        loss_bag = bce(preds[0], attrs[:,2].unsqueeze(1).to(torch.float32))

        #precision0 = torch.exp(-self.log_vars[0])
        #loss0 = precision0*loss0 + self.log_vars[0]

        # precision1 = torch.exp(-self.log_vars[1])
        # loss1 = precision1*loss1 + self.log_vars[1]

        # precision2 = torch.exp(-self.log_vars[2])
        # loss2 = precision2*loss2 + self.log_vars[2]
        
        return loss_bag + loss_backpack


def get_cost_function():
    # cost_function = torch.nn.CrossEntropyLoss()
    #cost_function = torch.nn.BCEWithLogitsLoss()
    return cost_function

def get_optimizer(model, lr, wd, momentum):
  
    # we will create two groups of weights, one for the newly initialized layer
    # and the other for rest of the layers of the network
    
    final_layer_weights = []
    rest_of_the_net_weights = []
    
    # we will iterate through the layers of the network
    for name, param in model.named_parameters():
        if name.startswith('classifier.6'):
            final_layer_weights.append(param)
        else:
            rest_of_the_net_weights.append(param)
    
    # so now we have divided the network weights into two groups.
    # We will train the final_layer_weights with learning_rate = lr
    # and rest_of_the_net_weights with learning_rate = lr / 10
    
    optimizer = torch.optim.SGD([
        {'params': rest_of_the_net_weights},
        {'params': final_layer_weights, 'lr': lr}
    ], lr=lr / 10, weight_decay=wd, momentum=momentum)
    
    return optimizer


def train(net,data_loader,optimizer,cost_function, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.
    cumulative_accuracy = 0.

    
    net.train() # Strictly needed if network contains layers which has different behaviours between train and test
    for batch_idx, (inputs,id, attr) in enumerate(data_loader):
    # for  (batch_idx,targets) in enumerate(data_loader):
        # print('inputs: ',targets[0].size(),'labels: ', targets[1].size())
        print('id: ', id)
        # t1 =id.to(device)

        # Load data into GPU
        inputs = inputs.to(device)
        # inputs = targets[0]
        targets = attr.to(device)
        # targets = targets[2]
        #print('targets: ',targets.size())
        
        # Forward pass
        outputs = net(inputs, get_features=False)
        #print('outputs: ',outputs)

        # Apply the loss
        # loss = cost_function(outputs,targets)
        loss = cost_function(outputs, targets) #.unsqueeze(1) or .reshape(-1,1)

        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        # Resets the gradients
        optimizer.zero_grad()

        # Better print something, no?
        samples+=inputs.shape[0]
        cumulative_loss += loss.item()
        # print(outputs[0])
        # print(outputs[0].squeeze(1))
        # print(outputs[0].squeeze(1).size())
        print(batch_idx)
        print(id)
        predicted = torch.round(outputs[0].squeeze(1))
        # try:
        #   print(predicted.size())
        #   print(t1.size())
        #   print(predicted.eq(t1))
        # except:
        #   print(targets[0].size())
        #   print(predicted.eq(targets[0]))

        cumulative_accuracy += predicted.sum()
        # cumulative_accuracy += predicted.eq(targets[1]).sum()

    return cumulative_loss/samples, cumulative_accuracy/samples*100

In [26]:
def main(train_loader,
         batch_size=128, 
         device='cuda:0', 
         learning_rate=0.001, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=50, 
         num_classes=27
         ):
  
  #   writer = SummaryWriter(log_dir="runs/exp1")

  # Instantiates dataloaders
  #   train_loader, test_loader = get_data(batch_size=batch_size, img_root=img_root)
  
  # Instantiates the model
  # net = initialize_resnet18(num_classes=num_classes).to(device)
  # net = FinetunedModel(architecture='resnet18',n_classes= 27).to(device)
  net = FinetunedModel(architecture='resnet50',n_classes= 27).to(device)
  
  # Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
  
  # Instantiates the cost function
  cost_function = MultiTaskLossWrapper(2)

  #   print('Before training:')
#   train_loss, train_accuracy = test(net, train_loader, cost_function)
#   test_loss, test_accuracy = test(net, test_loader, cost_function)

#   print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
#   print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
#   print('-----------------------------------------------------')
  
  # Add values to plots
#   writer.add_scalar('Loss/train_loss', train_loss, 0)
#   writer.add_scalar('Loss/test_loss', test_loss, 0)
#   writer.add_scalar('Accuracy/train_accuracy', train_accuracy, 0)
#   writer.add_scalar('Accuracy/test_accuracy', test_accuracy, 0)

  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
    # test_loss, test_accuracy = test(net, test_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    # print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')
    
    # Add values to plots
    # writer.add_scalar('Loss/train_loss', train_loss, e + 1)
    # writer.add_scalar('Loss/test_loss', test_loss, e + 1)
    # writer.add_scalar('Accuracy/train_accuracy', train_accuracy, e + 1)
    # writer.add_scalar('Accuracy/test_accuracy', test_accuracy, e + 1)

  print('After training:')
  # train_loss, train_accuracy = test(net, train_loader, cost_function)
#   test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
#   print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  # Closes the logger
#   writer.close()

In [27]:
# import numpy as np
# print(train_dataset.attr_df.dtypes)
# for column in train_dataset.attr_df.columns:
#       if(column!='age'):
#         train_dataset.attr_df[column] =np.array((train_dataset.attr_df[column].astype('str').replace({'1': '0', '2': '1'})).astype("int64"))
# print(train_dataset.attr_df.dtypes)


In [28]:
# #type(train_dataset.attr_df.age[0])
# for column in train_dataset.attr_df.columns:
#   train_dataset[column] = train_dataset[column].astype(int64)
# train_dataset.attr_df.dtypes

In [29]:
main(train_loader,
        batch_size=32,
        device='cuda:0', 
         learning_rate=0.001, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=1, 
         num_classes=27
         )

Backbone feature size: 2048


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


id:  ('0863', '0669', '0700', '0474', '1213', '0461', '0451', '1232')
0
('0863', '0669', '0700', '0474', '1213', '0461', '0451', '1232')
id:  ('0853', '0336', '0786', '1418', '0205', '0254', '0492', '0650')
1
('0853', '0336', '0786', '1418', '0205', '0254', '0492', '0650')
id:  ('0856', '1069', '0081', '1288', '0337', '1194', '1185', '0134')
2
('0856', '1069', '0081', '1288', '0337', '1194', '1185', '0134')
id:  ('1429', '1301', '1232', '1026', '0473', '0691', '0147', '0136')
3
('1429', '1301', '1232', '1026', '0473', '0691', '0147', '0136')
id:  ('0113', '0469', '0974', '1085', '0700', '1161', '1434', '0432')
4
('0113', '0469', '0974', '1085', '0700', '1161', '1434', '0432')
id:  ('1354', '0752', '1035', '1178', '0630', '0398', '0766', '0485')
5
('1354', '0752', '1035', '1178', '0630', '0398', '0766', '0485')
id:  ('0150', '1461', '1225', '0361', '0830', '0593', '0969', '1089')
6
('0150', '1461', '1225', '0361', '0830', '0593', '0969', '1089')
id:  ('1481', '0465', '0338', '0586', '03

In [16]:
attr = train_dataset.attr_df[train_dataset.attr_df["id"] == int("1461")].values[0][1:]
attr

array([2, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0])

In [ ]:
input = image in for of 
target (ground_truth)= {0,1,3,1,0,1,0,1,0} -> (N,C)
output (predicted) = {1,1,2,0,1,1,1,0,1} -> single value:  {1}, {1}

loss = 0
for value in output:
  loss += crossEntropy(value,target)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

print(input)
print(target)
print(output)

tensor([[-1.3493, -0.6075,  0.5715,  1.5922, -1.1586],
        [-0.8425,  1.0423,  0.5128,  1.9887, -0.6489],
        [-0.9244,  1.4074, -0.8986,  2.1912, -0.3142]], requires_grad=True)
tensor([3, 1, 3])
tensor(0.8181, grad_fn=<NllLossBackward>)


In [ ]:
train_loader -> (X, y, attr)
loss = 0
target -> true value
input -> predicted

outputs[idx] -> shape[batch, n_classes]
value.attr[idx] -> ground truth (true label)
for idx, value in enumerate(train_loader):
  loss += loss_func(outputs[idx], value.attr[idx])
  loss += train_loader.attr[1]

# NOT HERE YET!

In [ ]:
from utils import image_utils
image_utils.imshow(train_dataset[10][0])
print(train_dataset[10])

In [ ]:
print(len(train_dataset.attr))
print(len(train_dataset.class_to_idx))

In [ ]:
from utils import image_utils
print(train_dataset.images_list[0])
print(image_utils.get_ids_from_images(train_dataset.images_list)[0])
image_utils.imshow(train_dataset[0][0])


In [ ]:
train_dataset.__init__(root_dir=config["train_root"],
                        attributes_file=config["attributes_file"], images_list=img_l).__getitem__()

In [ ]:
img_l = train_dataset.images_list

In [ ]:
train_dataset.__getitem__(int('0901'))

In [ ]:
train_dataset.__getitem__(int('0901'))

In [ ]:
idx = train_dataset.class_to_idx.keys()
# idx
train_dataset.__getitem__(int('0901'))

In [ ]:
train_dataset